We can use Neo4j to map out stations, the zip codes that are within 1 mile radius, and map the most efficient route for delivery within 1 mile radius.
* Public transit to transport deliveries: Time to make deliveries and which would be the shortest or fastest route
* Pick up: which stations would potentially serve the greatest number of customers
* Delivery drones/hybrid: Shortest route to make the delivery from the Bart station

Delivery Drones: 1 mile radius.

In [1]:
!pip install folium

In [2]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

from IPython.display import display

import folium

# from folium import Map
from folium.plugins import HeatMap

In [3]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [4]:
session = driver.session(database="neo4j")

In [5]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [6]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [7]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [8]:
cursor = connection.cursor()

In [9]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [10]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [11]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

In [12]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [13]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [14]:
def my_neo4j_create_station_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [15]:
def my_neo4j_create_zip_node(zip_code):
    "create a node with label Zip_Code"
    
    query = """
    
    CREATE (:Zip_Code {name: $zip_code})
    
    """
    
    session.run(query, zip_code=zip_code)
    

In [16]:
# def my_neo4j_create_relationship_one_way(from_zip, to_station, weight):
#     "create a relationship one way from a zip code to a station with a weight"
    
#     query = """
    
#     MATCH (from:Zip_Code), 
#           (to:Station)
#     WHERE from.name = $from_zip and to.name = $to_station
#     CREATE (from)-[:LINK {weight: $weight}]->(to)
    
#     """

#     session.run(query, from_zip=from_zip, to_station=to_station, weight=weight)
    

In [17]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way from a station to a station with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """

    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [18]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [19]:
def my_pickuplocat_get_zips(pickuplocat, miles,customer=False):
    "given a pickup locatation, pull all zip codes with miles distance, print them, sum the population and customers"
    
    connection.rollback()
    
    query = "select latitude, longitude from "
    query += "(select station,latitude, longitude from stations "
    query += "union select concat(city,' store') as station, latitude, longitude from stores) locat_gps "
    query += "where station = '" + pickuplocat + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select z.zip, z.population "
    if customer == True:
        query += " , count(c.customer_id) as cnt "
    query += " from zip_codes as z "
    if customer == True:
        query += " join customers as c on z.zip=c.zip "
    query += " where z.latitude >= " + str(bottom[0])
    query += " and z.latitude <= " + str(top [0])
    query += " and z.longitude >= " + str(left[1])
    query += " and z.longitude <= " + str(right[1])
    if customer == True:
        query += " group by z.zip,z.population "
    query += " order by 1 "
    
    

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + pickuplocat + " Pickup Location")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    if customer==True:
        total_customers = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        if customer==False:
            print("     zip:", zip, "  population: ", f'{population:10,}')
        if customer==True:
            customers = row[2]
            print("     zip:", zip, "  population: ", f'{population:10,}', "  customer: ", f'{customers:10,}')
            total_customers += customers
        
        total_population += population
        
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    if customer==True:
        print("  Total Customers: ", f'{total_customers:10,}')
    print("-------------------------------------------------------------------------------")

my_pickuplocat_get_zips("Ashby",1,True)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Ashby Pickup Location
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092   customer:         171
     zip: 94703   population:      21,937   customer:         155
     zip: 94704   population:      29,190   customer:         161

-------------------------------------------------------------------------------
  Total Population:      68,219
  Total Customers:         487
-------------------------------------------------------------------------------


We have a branch in Berkeley. How many customers does it serve?

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select latitude, longitude from
    (select station,latitude, longitude from stations
    union select city as station, latitude, longitude from stores) as locat_gps
    where station = 'Berkeley'

"""

locats = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
locats

,latitude,longitude
0,37.8555,-122.2604


In [21]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station, line
from lines
order  by station, line

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,12th Street,orange
1,12th Street,red
2,12th Street,yellow
3,16th Street Mission,blue
4,16th Street Mission,green
...,...,...
109,West Dublin,blue
110,West Oakland,blue
111,West Oakland,green
112,West Oakland,red


In [22]:
# rollback_before_flag = True
# rollback_after_flag = True

# query = """

# select count(*) as cnt
# from customers


# """

# my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

In [23]:
# # Get table of customers within 1 mile of Bart station and Berkeley store.
# # Customer identifier, address (lon, lat). pickup location associated with customer

# # drop this new customer table if it exists
# connection.rollback()

# query = """

# DROP TABLE IF EXISTS customer_locat;

# """

# cursor.execute(query)

# connection.commit()

# # Create the customer_locat table

# connection.rollback()

# query = """

# CREATE TABLE customer_locat(
#     customer_id VARCHAR(32) PRIMARY KEY,
#     latitude NUMERIC(9,6),
#     longitude NUMERIC(9,6),
#     zip VARCHAR(5)
# )

# """

# cursor.execute(query)

# connection.commit()

# ###### insert into the customer_locat table

# connection.rollback()

# query = "select latitude, longitude from "
# query += "(select station,latitude, longitude from stations "
# query += "union select concat(city,' store') as station, latitude, longitude from stores) locat_gps "

# cursor.execute(query)

# connection.rollback()

# rows = cursor.fetchall()


# for row in rows:
#     latitude = row[0]
#     longitude = row[1]
#     point = (latitude, longitude)
#     (left, right, top, bottom) = my_calculate_box(point, 1)

#     query = "insert into customer_locat (customer_id, latitude, longitude, zip) "
#     query += " select c.customer_id, z.latitude, z. longitude, c.zip "
#     query += " from zip_codes as z "
#     query += " join customers as c on z.zip=c.zip "
#     query += " where z.latitude >= " + str(bottom[0])
#     query += " and z.latitude <= " + str(top [0])
#     query += " and z.longitude >= " + str(left[1])
#     query += " and z.longitude <= " + str(right[1])
#     query += " order by 1;"
#     connection.commit()
#     connection.rollback()

# rollback_before_flag = True
# rollback_after_flag = True

# query = """

# select *
# from customer_locat
# limit 10

# """

# my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)


In [24]:
my_neo4j_wipe_out_database()

In [25]:
# customer=True

# connection.rollback()

# query = "select latitude, longitude, station from "
# query += "(select station,latitude, longitude from stations "
# query += "union select concat(city,' store') as station, latitude, longitude from stores where city = 'Berkeley') locat_gps"

# cursor.execute(query)

# connection.rollback()

# rows = cursor.fetchall()

# for row in rows:
#     latitude = row[0]
#     longitude = row[1]
#     station = row[2]
#     my_neo4j_create_station_node(station)

#     point = (latitude, longitude)

#     (left, right, top, bottom) = my_calculate_box(point, 1)

#     query = "select z.zip, z.population "
#     if customer == True:
#         query += " , count(c.customer_id) as cnt "
#     query += " from zip_codes as z "
#     if customer == True:
#         query += " join customers as c on z.zip=c.zip "
#     query += " where z.latitude >= " + str(bottom[0])
#     query += " and z.latitude <= " + str(top [0])
#     query += " and z.longitude >= " + str(left[1])
#     query += " and z.longitude <= " + str(right[1])
#     if customer == True:
#         query += " group by z.zip,z.population "
#     query += " order by 1 "

#     cursor.execute(query)

#     connection.rollback()

#     rows = cursor.fetchall()


#     total_population = 0
#     if customer==True:
#         total_customers = 0

#     for row in rows:
#         zip_code = row[0]
#         my_neo4j_create_zip_node(zip_code)
#         population = row[1]
#         if customer==True:
#             customers = row[2]
#             total_customers += customers

#         total_population += population
        
#         if customer==False:
#             my_neo4j_create_relationship_one_way(zip_code,station, int(total_population))
#         if customer==True:
#             my_neo4j_create_relationship_one_way(zip_code,station, int(total_customers))

# my_neo4j_create_relationship_two_way("Dublin","West Dublin",0)
# my_neo4j_create_relationship_two_way("Dublin","Castro Valley",0)
# my_neo4j_create_relationship_two_way("Castro Valley","Bay Fair",0)
# my_neo4j_create_relationship_two_way("Bay Fair","San Leandro",0)
# my_neo4j_create_relationship_two_way("San Leandro","Coliseum",0)
# my_neo4j_create_relationship_two_way("Berryessa","Milpitas",0)
# my_neo4j_create_relationship_two_way("Milpitas","Warm Springs",0)
# my_neo4j_create_relationship_two_way("Warm Springs","Fremont",0)
# my_neo4j_create_relationship_two_way("Fremont","Union City",0)
# my_neo4j_create_relationship_two_way("Union City","South Hayward",0)
# my_neo4j_create_relationship_two_way("South Hayward","Hayward",0)
# my_neo4j_create_relationship_two_way("Hayward","Bay Fair",0)
# my_neo4j_create_relationship_two_way("Coliseum","OAK",0)
# my_neo4j_create_relationship_two_way("Coliseum","Fruitvale",0)
# my_neo4j_create_relationship_two_way("Fruitvale","Lake Merritt",0)
# my_neo4j_create_relationship_two_way("Lake Merritt","12th Street",0)
# my_neo4j_create_relationship_two_way("12th Street","19th Street",0)
# my_neo4j_create_relationship_two_way("19th Street","MacArthur",0)
# my_neo4j_create_relationship_two_way("MacArthur","Ashby",0)
# my_neo4j_create_relationship_two_way("Ashby","Downtown Berkeley",0)
# my_neo4j_create_relationship_two_way("Downtown Berkeley","North Berkeley",0)
# my_neo4j_create_relationship_two_way("North Berkeley","El Cerrito Plaza",0)
# my_neo4j_create_relationship_two_way("El Cerrito Plaza","El Cerrito del Norte",0)
# my_neo4j_create_relationship_two_way("El Cerrito del Norte","Richmond",0)
# my_neo4j_create_relationship_two_way("MacArthur","Rockridge",0)
# my_neo4j_create_relationship_two_way("Rockridge","Orinda",0)
# my_neo4j_create_relationship_two_way("Orinda","Lafayette",0)
# my_neo4j_create_relationship_two_way("Lafayette","Walnut Creek",0)
# my_neo4j_create_relationship_two_way("Walnut Creek","Pleasant Hill",0)
# my_neo4j_create_relationship_two_way("Pleasant Hill","Concord",0)
# my_neo4j_create_relationship_two_way("Concord","North Concord",0)
# my_neo4j_create_relationship_two_way("North Concord","Pittsburg",0)
# my_neo4j_create_relationship_two_way("Pittsburg","Pittsburg Center",0)
# my_neo4j_create_relationship_two_way("Pittsburg Center","Antioch",0)
# my_neo4j_create_relationship_two_way("Lake Merritt","West Oakland",0)
# my_neo4j_create_relationship_two_way("West Oakland","Embarcadero",0)
# my_neo4j_create_relationship_two_way("Embarcadero","Montgomery Street",0)
# my_neo4j_create_relationship_two_way("Montgomery Street","Powell Street",0)
# my_neo4j_create_relationship_two_way("Powell Street","Civic Center",0)
# my_neo4j_create_relationship_two_way("Civic Center","16th Street Mission",0)
# my_neo4j_create_relationship_two_way("16th Street Mission","24th Street Mission",0)
# my_neo4j_create_relationship_two_way("24th Street Mission","Glen Park",0)
# my_neo4j_create_relationship_two_way("Glen Park","Balboa Park",0)
# my_neo4j_create_relationship_two_way("Balboa Park","Daly City",0)
# my_neo4j_create_relationship_two_way("Daly City","Colma",0)
# my_neo4j_create_relationship_two_way("Colma","South San Francisco",0)
# my_neo4j_create_relationship_two_way("South San Francisco","San Bruno",0)
# my_neo4j_create_relationship_two_way("San Bruno","Millbrae",0)
# my_neo4j_create_relationship_two_way("Millbrae","SFO",0)
# my_neo4j_create_relationship_two_way("Downtown Berkeley","Berkeley store",0)
# my_neo4j_create_relationship_two_way("North Berkeley","Berkeley store",0)

In [26]:
customer=True

connection.rollback()

station_list = []
pop_counts = []
cust_1mi_counts = []

query = "select latitude, longitude, station from "
query += "(select station,latitude, longitude from stations "
query += "union select concat(city,' store') as station, latitude, longitude from stores where city = 'Berkeley') locat_gps"

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    latitude = row[0]
    longitude = row[1]
    station = row[2]
    station_list.append(station)
    my_neo4j_create_station_node(station)

    point = (latitude, longitude)

    (left, right, top, bottom) = my_calculate_box(point, 1)

    query = "select z.zip, z.population "
    if customer == True:
        query += " , count(c.customer_id) as cnt "
    query += " from zip_codes as z "
    if customer == True:
        query += " join customers as c on z.zip=c.zip "
    query += " where z.latitude >= " + str(bottom[0])
    query += " and z.latitude <= " + str(top [0])
    query += " and z.longitude >= " + str(left[1])
    query += " and z.longitude <= " + str(right[1])
    if customer == True:
        query += " group by z.zip,z.population "
    query += " order by 1 "

    cursor.execute(query)

    connection.rollback()

    rows = cursor.fetchall()


    total_population = 0
    if customer==True:
        total_customers = 0

    for row in rows:
        zip_code = row[0]
#         my_neo4j_create_zip_node(zip_code)
        population = row[1]
        if customer==True:
            customers = row[2]
            total_customers += customers
        total_population += population
    pop_counts.append(int(total_population))
    cust_1mi_counts.append(total_customers)
        
#         if customer==False:
#             my_neo4j_create_relationship_one_way(zip_code,station, int(total_population))
#         if customer==True:
#             my_neo4j_create_relationship_one_way(zip_code,station, int(total_customers))

# my_neo4j_create_relationship_two_way("Dublin","West Dublin",0)
# my_neo4j_create_relationship_two_way("Dublin","Castro Valley",0)
# my_neo4j_create_relationship_two_way("Castro Valley","Bay Fair",0)
# my_neo4j_create_relationship_two_way("Bay Fair","San Leandro",0)
# my_neo4j_create_relationship_two_way("San Leandro","Coliseum",0)
# my_neo4j_create_relationship_two_way("Berryessa","Milpitas",0)
# my_neo4j_create_relationship_two_way("Milpitas","Warm Springs",0)
# my_neo4j_create_relationship_two_way("Warm Springs","Fremont",0)
# my_neo4j_create_relationship_two_way("Fremont","Union City",0)
# my_neo4j_create_relationship_two_way("Union City","South Hayward",0)
# my_neo4j_create_relationship_two_way("South Hayward","Hayward",0)
# my_neo4j_create_relationship_two_way("Hayward","Bay Fair",0)
# my_neo4j_create_relationship_two_way("Coliseum","OAK",0)
# my_neo4j_create_relationship_two_way("Coliseum","Fruitvale",0)
# my_neo4j_create_relationship_two_way("Fruitvale","Lake Merritt",0)
# my_neo4j_create_relationship_two_way("Lake Merritt","12th Street",0)
# my_neo4j_create_relationship_two_way("12th Street","19th Street",0)
# my_neo4j_create_relationship_two_way("19th Street","MacArthur",0)
# my_neo4j_create_relationship_two_way("MacArthur","Ashby",0)
# my_neo4j_create_relationship_two_way("Ashby","Downtown Berkeley",0)
# my_neo4j_create_relationship_two_way("Downtown Berkeley","North Berkeley",0)
# my_neo4j_create_relationship_two_way("North Berkeley","El Cerrito Plaza",0)
# my_neo4j_create_relationship_two_way("El Cerrito Plaza","El Cerrito del Norte",0)
# my_neo4j_create_relationship_two_way("El Cerrito del Norte","Richmond",0)
# my_neo4j_create_relationship_two_way("MacArthur","Rockridge",0)
# my_neo4j_create_relationship_two_way("Rockridge","Orinda",0)
# my_neo4j_create_relationship_two_way("Orinda","Lafayette",0)
# my_neo4j_create_relationship_two_way("Lafayette","Walnut Creek",0)
# my_neo4j_create_relationship_two_way("Walnut Creek","Pleasant Hill",0)
# my_neo4j_create_relationship_two_way("Pleasant Hill","Concord",0)
# my_neo4j_create_relationship_two_way("Concord","North Concord",0)
# my_neo4j_create_relationship_two_way("North Concord","Pittsburg",0)
# my_neo4j_create_relationship_two_way("Pittsburg","Pittsburg Center",0)
# my_neo4j_create_relationship_two_way("Pittsburg Center","Antioch",0)
# my_neo4j_create_relationship_two_way("Lake Merritt","West Oakland",0)
# my_neo4j_create_relationship_two_way("West Oakland","Embarcadero",0)
# my_neo4j_create_relationship_two_way("Embarcadero","Montgomery Street",0)
# my_neo4j_create_relationship_two_way("Montgomery Street","Powell Street",0)
# my_neo4j_create_relationship_two_way("Powell Street","Civic Center",0)
# my_neo4j_create_relationship_two_way("Civic Center","16th Street Mission",0)
# my_neo4j_create_relationship_two_way("16th Street Mission","24th Street Mission",0)
# my_neo4j_create_relationship_two_way("24th Street Mission","Glen Park",0)
# my_neo4j_create_relationship_two_way("Glen Park","Balboa Park",0)
# my_neo4j_create_relationship_two_way("Balboa Park","Daly City",0)
# my_neo4j_create_relationship_two_way("Daly City","Colma",0)
# my_neo4j_create_relationship_two_way("Colma","South San Francisco",0)
# my_neo4j_create_relationship_two_way("South San Francisco","San Bruno",0)
# my_neo4j_create_relationship_two_way("San Bruno","Millbrae",0)
# my_neo4j_create_relationship_two_way("Millbrae","SFO",0)
# my_neo4j_create_relationship_two_way("Downtown Berkeley","Berkeley store",0)
# my_neo4j_create_relationship_two_way("North Berkeley","Berkeley store",0)

In [27]:
stations_cust = pd.DataFrame(list(zip(station_list, pop_counts, cust_1mi_counts)), 
                             columns = ["Station", "Population", "Customers"])
stations_cust.sort_values(by="Station")

,Station,Population,Customers
49,12th Street,16062,161
23,16th Street Mission,63489,135
31,19th Street,16062,161
16,24th Street Mission,108915,133
11,Antioch,0,0
22,Ashby,68219,487
14,Balboa Park,106589,76
20,Bay Fair,41059,63
21,Berkeley store,51127,316
47,Berryessa,0,0


In [28]:
my_neo4j_create_relationship_one_way("Antioch","Pittsburg Center",0)

my_neo4j_create_relationship_one_way("Pittsburg Center","Pittsburg",1)
my_neo4j_create_relationship_one_way("Pittsburg Center","Antioch",1)

my_neo4j_create_relationship_one_way("Pittsburg","Pittsburg Center",0)
my_neo4j_create_relationship_one_way("Pittsburg","North Concord",0)

my_neo4j_create_relationship_one_way("North Concord","Pittsburg",35)
my_neo4j_create_relationship_one_way("North Concord","Concord",35)

my_neo4j_create_relationship_one_way("Concord","North Concord",0)
my_neo4j_create_relationship_one_way("Concord","Pleasant Hill",0)

my_neo4j_create_relationship_one_way("Pleasant Hill","Concord",74)
my_neo4j_create_relationship_one_way("Pleasant Hill","Walnut Creek",74)

my_neo4j_create_relationship_one_way("Walnut Creek","Pleasant Hill",74)
my_neo4j_create_relationship_one_way("Walnut Creek","Lafayette",74)

my_neo4j_create_relationship_one_way("Lafayette","Walnut Creek",129)
my_neo4j_create_relationship_one_way("Lafayette","Orinda",129)

my_neo4j_create_relationship_one_way("Orinda","Lafayette",193)
my_neo4j_create_relationship_one_way("Orinda","Rockridge",193)

my_neo4j_create_relationship_one_way("Rockridge","Orinda",311)
my_neo4j_create_relationship_one_way("Rockridge","MacArthur",311)

my_neo4j_create_relationship_one_way("MacArthur","Rockridge",155)
my_neo4j_create_relationship_one_way("MacArthur","19th Street",155)
my_neo4j_create_relationship_one_way("MacArthur","Ashby",155)

my_neo4j_create_relationship_one_way("19th Street","MacArthur",161)
my_neo4j_create_relationship_one_way("19th Street","12th Street",161)

my_neo4j_create_relationship_one_way("12th Street","19th Street",161)
my_neo4j_create_relationship_one_way("12th Street","Lake Merritt",161)
my_neo4j_create_relationship_one_way("12th Street","West Oakland",161)

my_neo4j_create_relationship_one_way("Lake Merritt","12th Street",161)
my_neo4j_create_relationship_one_way("Lake Merritt","Fruitvale",161)
my_neo4j_create_relationship_one_way("Lake Merritt","West Oakland",161)

my_neo4j_create_relationship_one_way("Fruitvale","Lake Merritt",180)
my_neo4j_create_relationship_one_way("Fruitvale","Coliseum",180)

my_neo4j_create_relationship_one_way("Coliseum","Fruitvale",0)
my_neo4j_create_relationship_one_way("Coliseum","OAK",0)
my_neo4j_create_relationship_one_way("Coliseum","San Leandro",0)

my_neo4j_create_relationship_one_way("OAK","Coliseum",0)

my_neo4j_create_relationship_one_way("San Leandro","Coliseum",72)
my_neo4j_create_relationship_one_way("San Leandro","Bay Fair",72)

my_neo4j_create_relationship_one_way("Bay Fair","San Leandro",63)
my_neo4j_create_relationship_one_way("Bay Fair","Castro Valley",63)
my_neo4j_create_relationship_one_way("Bay Fair","Hayward",63)

my_neo4j_create_relationship_one_way("Castro Valley","Bay Fair",0)
my_neo4j_create_relationship_one_way("Castro Valley","West Dublin",0)

my_neo4j_create_relationship_one_way("West Dublin","Dublin",0)
my_neo4j_create_relationship_one_way("West Dublin","Castro Valley",0)

my_neo4j_create_relationship_one_way("Dublin","West Dublin",10)

my_neo4j_create_relationship_one_way("Hayward","Bay Fair",10)
my_neo4j_create_relationship_one_way("Hayward","South Hayward",10)

my_neo4j_create_relationship_one_way("South Hayward","Hayward",9)
my_neo4j_create_relationship_one_way("South Hayward","Union City",9)

my_neo4j_create_relationship_one_way("Union City","South Hayward",9)
my_neo4j_create_relationship_one_way("Union City","Fremont",9)

my_neo4j_create_relationship_one_way("Fremont","Union City",8)
my_neo4j_create_relationship_one_way("Fremont","Warm Springs",8)

my_neo4j_create_relationship_one_way("Warm Springs","Fremont",0)
my_neo4j_create_relationship_one_way("Warm Springs","Milpitas",0)

my_neo4j_create_relationship_one_way("Milpitas","Warm Springs",0)
my_neo4j_create_relationship_one_way("Milpitas","Berryessa",0)

my_neo4j_create_relationship_one_way("Berryessa","Milpitas",0)

my_neo4j_create_relationship_one_way("Ashby","MacArthur",487)
my_neo4j_create_relationship_one_way("Ashby","Downtown Berkeley",487)

my_neo4j_create_relationship_one_way("Downtown Berkeley","Ashby",634)
my_neo4j_create_relationship_one_way("Downtown Berkeley","North Berkeley",634)

my_neo4j_create_relationship_one_way("North Berkeley","Downtown Berkeley",443)
my_neo4j_create_relationship_one_way("North Berkeley","El Cerrito Plaza",443)

my_neo4j_create_relationship_one_way("El Cerrito Plaza","El Cerrito del Norte",175)
my_neo4j_create_relationship_one_way("El Cerrito Plaza","North Berkeley",175)

my_neo4j_create_relationship_one_way("El Cerrito del Norte","El Cerrito Plaza",0)
my_neo4j_create_relationship_one_way("El Cerrito del Norte","Richmond",0)

my_neo4j_create_relationship_one_way("Richmond","El Cerrito del Norte",0)

my_neo4j_create_relationship_one_way("West Oakland","12th Street",203)
my_neo4j_create_relationship_one_way("West Oakland","Lake Merritt",203)
my_neo4j_create_relationship_one_way("West Oakland","Embarcadero",203)

my_neo4j_create_relationship_one_way("Embarcadero","West Oakland",196)
my_neo4j_create_relationship_one_way("Embarcadero","Montgomery Street",196)

my_neo4j_create_relationship_one_way("Montgomery Street","Embarcadero",274)
my_neo4j_create_relationship_one_way("Montgomery Street","Powell Street",274)

my_neo4j_create_relationship_one_way("Powell Street","Montgomery Street",333)
my_neo4j_create_relationship_one_way("Powell Street","Civic Center",333)

my_neo4j_create_relationship_one_way("Civic Center","Powell Street",210)
my_neo4j_create_relationship_one_way("Civic Center","16th Street Mission",210)

my_neo4j_create_relationship_one_way("16th Street Mission","Civic Center",135)
my_neo4j_create_relationship_one_way("16th Street Mission","24th Street Mission",135)

my_neo4j_create_relationship_one_way("24th Street Mission","16th Street Mission",133)
my_neo4j_create_relationship_one_way("24th Street Mission","Glen Park",133)

my_neo4j_create_relationship_one_way("Glen Park","24th Street Mission",103)
my_neo4j_create_relationship_one_way("Glen Park","Balboa Park",103)

my_neo4j_create_relationship_one_way("Balboa Park","Glen Park",76)
my_neo4j_create_relationship_one_way("Balboa Park","Daly City",76)

my_neo4j_create_relationship_one_way("Daly City","Balboa Park",0)
my_neo4j_create_relationship_one_way("Daly City","Colma",0)

my_neo4j_create_relationship_one_way("Colma","Daly City",4)
my_neo4j_create_relationship_one_way("Colma","South San Francisco",4)

my_neo4j_create_relationship_one_way("South San Francisco","Colma",0)
my_neo4j_create_relationship_one_way("South San Francisco","San Bruno",0)

my_neo4j_create_relationship_one_way("San Bruno","South San Francisco",9)
my_neo4j_create_relationship_one_way("San Bruno","SFO",9)
my_neo4j_create_relationship_one_way("San Bruno","Millbrae",9)

my_neo4j_create_relationship_one_way("SFO","San Bruno",1)
my_neo4j_create_relationship_one_way("SFO","Millbrae",1)

my_neo4j_create_relationship_one_way("Millbrae","San Bruno",10)
my_neo4j_create_relationship_one_way("Millbrae","SFO",10)


In [29]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 51
  Relationships: 102
-------------------------


In [30]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [31]:
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,12th Street,[Station]
1,16th Street Mission,[Station]
2,19th Street,[Station]
3,24th Street Mission,[Station]
4,Antioch,[Station]
5,Ashby,[Station]
6,Balboa Park,[Station]
7,Bay Fair,[Station]
8,Berkeley store,[Station]
9,Berryessa,[Station]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,12th Street,[Station],LINK,19th Street,[Station]
1,12th Street,[Station],LINK,Lake Merritt,[Station]
2,12th Street,[Station],LINK,West Oakland,[Station]
3,16th Street Mission,[Station],LINK,24th Street Mission,[Station]
4,16th Street Mission,[Station],LINK,Civic Center,[Station]
...,...,...,...,...,...
97,West Dublin,[Station],LINK,Castro Valley,[Station]
98,West Dublin,[Station],LINK,Dublin,[Station]
99,West Oakland,[Station],LINK,12th Street,[Station]
100,West Oakland,[Station],LINK,Embarcadero,[Station]


-------------------------
  Density: 0.1
-------------------------


In [32]:
# Which of the stations are best for delivery?

In [33]:
zips_1mi = []
lat_1mi = []
lon_1mi = []


connection.rollback()

query = "select latitude, longitude from "
query += "(select station,latitude, longitude from stations "
query += "union select concat(city,' store') as station, latitude, longitude from stores) locat_gps "

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()


for i in rows:
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]

        point = (latitude, longitude)

        (left, right, top, bottom) = my_calculate_box(point, 1)

        query = "select z.zip, z.latitude, z.longitude"
        query += " from zip_codes as z "
        query += " where z.latitude >= " + str(bottom[0])
        query += " and z.latitude <= " + str(top [0])
        query += " and z.longitude >= " + str(left[1])
        query += " and z.longitude <= " + str(right[1])
        query += " order by 1 "

        cursor.execute(query)

        connection.rollback()

        rows = cursor.fetchall()
        if rows != []:
            for row in rows:
                zips_1mi.append(row[0])
                lat_1mi.append(row[1])
                lon_1mi.append(row[2])

In [41]:
query = """

CALL gds.pageRank.stream('ds_graph',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor,
                           relationshipWeightProperty: 'weight'}
                         )
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS station, score as page_rank
ORDER BY page_rank DESC, station ASC

"""




max_iterations = 20
damping_factor = 0.05

my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)



,station,page_rank
0,MacArthur,1.024200
1,West Oakland,1.008395
2,12th Street,1.008200
3,Lake Merritt,1.007780
4,Civic Center,1.000000
5,16th Street Mission,1.000000
6,Powell Street,0.999995
7,24th Street Mission,0.999984
8,Lafayette,0.999979
9,Orinda,0.999801


In [35]:
# creating the customers density heat map
connection.rollback()

query = """

drop table if exists zips_1mi;

"""

cursor.execute(query)

connection.commit()

connection.rollback()

query = """

CREATE TABLE zips_1mi(
    zips varchar,
    latitude varchar,
    longitude varchar
)

"""

cursor.execute(query)

connection.commit()

for i in range(len(zips_1mi)):
    connection.rollback()
    cursor.execute("insert into zips_1mi (zips, latitude, longitude) values (%s,%s,%s);", 
                   (zips_1mi[i],lat_1mi[i],lon_1mi[i]))
    connection.commit()

connection.rollback()

rollback_before_flag = True
rollback_after_flag = True


query = """

select z.latitude, z.longitude
from customers as c
join zips_1mi as z on c.zip=z.zips

"""

locats = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

locats


,latitude,longitude
0,37.7797,-122.4192
1,37.7797,-122.4192
2,37.7797,-122.4192
3,37.7797,-122.4192
4,37.7797,-122.4192
...,...,...
6740,37.8088,-122.2691
6741,37.8088,-122.2691
6742,37.8088,-122.2691
6743,37.8088,-122.2691


In [36]:
# map marker of bart stations

rollback_before_flag = True
rollback_after_flag = True


query = """

select station, latitude, longitude
from stations

"""

station_gps = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
station_gps

,station,latitude,longitude
0,12th Street,37.803608,-122.272006
1,16th Street Mission,37.764847,-122.420042
2,19th Street,37.807869,-122.268980
3,24th Street Mission,37.752000,-122.418700
4,Antioch,37.996281,-121.783404
5,Ashby,37.853068,-122.269957
6,Balboa Park,37.721667,-122.447500
7,Bay Fair,37.697000,-122.126500
8,Berryessa,37.368361,-121.874655
9,Castro Valley,37.690748,-122.075679


In [37]:
# map of the Bart station and 1 mile radius. around the pick up stations and the Berekeley store.
# https://techwetrust.com/how-to/python/how-to-make-a-geographical-heat-map-with-python/

for_map = folium.Map(location = [37.8555,-122.2604], zoom_start=10,)
hm_wide = HeatMap(
    list(zip(locats.latitude.values, locats.longitude.values)),
    min_opacity=0.2,
    radius=17,
    blur=15,
    max_zoom=1
)
for i in range(len(station_gps)):
    folium.Marker(
        location = [station_gps.iloc[i]['latitude'], station_gps.iloc[i]["longitude"]],
        popup=station_gps.iloc[i]["station"],
        icon = folium.DivIcon(html="""
            <div><svg>
                <circle cx="5" cy="5" r="5" fill="#170918" opacity="1"/>
            </svg></div>
        """)
    ).add_to(for_map)

for_map.add_child(hm_wide)
